In [ ]:
import sys
import yaml
import itertools
import subprocess
import multiprocessing

In [ ]:
from pathlib import Path

# Compile original Lyra2 implementations

Need to import python functions that compile Lyra from a git submodule. Manipulate path as per https://stackoverflow.com/a/29747054/1269892

In [ ]:
sys.path.append(str(Path('../lyra/Lyra2/tests').resolve()))

In [ ]:
from harness import build_lyra2, unlist_values
from harness import compose_sponge_name, compose_lyra2_name

In [ ]:
with open('harness.yml', 'r') as config:
    params = yaml.load(config)

In [ ]:
build_lyra2(params)

In [ ]:
build_path0 = Path('./bin42').resolve()
if not build_path0.exists():
    print('Could not find ./bin42, directory for original executables')

# Compile ported Java implementation

In [ ]:
subprocess.run(['mvn', 'package', '-f', '../lyra2-java', '-Plyra2-compare', '-DskipTests'])

In [ ]:
build_path1 = Path('./target').resolve()
if not build_path1.exists():
    print('Could not ./target, directory for compiled executables')

# Class to measure time and memory usage

Resources used for memory measurements: https://stackoverflow.com/questions/22372960/is-this-explanation-about-vss-rss-pss-uss-accurate

In [ ]:
import time
import psutil

In [ ]:
from statistics import median

In [ ]:
class ProcessObserver:
    def __init__(self, ntimes=3, mtimes=3):
        # Number of times to measure elapsed time
        self.ntimes = ntimes
        # Number of times to measure consumed memory
        self.mtimes = mtimes

    def run(self, *args):
        # Run the process self.ntimes and see how long it takes
        times = [-1 for i in range(self.ntimes)]
        for i in range(self.ntimes):
            fst = time.time()
            
            process = subprocess.run(*args)
                        
            lst = time.time()
            
            if process.returncode != 0:
                print(args[0][0] + ' failed to run')

                continue
            
            elapsed = lst - fst # seconds
                
            if elapsed > 300:
                print(args[0][0] + ' has been running for: ' + elapsed)
            
            times[i] = elapsed
            
        mtime = median(times)
        
        process = subprocess.Popen(*args)
        
        # Approximate running time of the process is known
        # The process has just been started asynchronously
        # Measure its memory usage (which is complicated)
        pss_mems = [-1 for i in range(self.mtimes)]
        uss_mems = [-1 for i in range(self.mtimes)]
        for i in range(self.mtimes):
            if process.poll() is not None:
                break # the process no longer runs, break

            # the process will finish when you least expect it
            # so wrap everything into try-except and handle it
            try:
                p = psutil.Process(process.pid)
                mem = p.memory_full_info()

                # total private memory + proporional size for the 3pp libraries
                pss_mems[i] = mem.pss
                # total private memory of a process (unique to the process)
                uss_mems[i] = mem.uss
            except: 
                break # the process no longer runs, break

            time.sleep(max(0, mtime / self.mtimes))
            
            
        return [median(times), max(pss_mems), max(uss_mems)]

# Prepare database schema

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy import Column, ForeignKey
from sqlalchemy import Integer, BigInteger, String, Float

In [ ]:
from sqlalchemy.sql import exists
from sqlalchemy.orm import sessionmaker, relationship
from sqlalchemy.orm.exc import NoResultFound
from sqlalchemy.ext.declarative import declarative_base

In [ ]:
Base = declarative_base()

In [ ]:
class Project(Base):
    __tablename__ = 'projects'
    
    id = Column(Integer, primary_key=True)
    project = Column(String)

class Password(Base):
    __tablename__ = 'passwords'
    
    id = Column(Integer, primary_key=True)
    password = Column(String)
    
class Salt(Base):
    __tablename__ = 'salts'
    
    id = Column(Integer, primary_key=True)
    salt = Column(String)
    
class Klen(Base):
    __tablename__ = 'klens'
    
    id = Column(Integer, primary_key=True)
    klen = Column(Integer)
    
class Block(Base):
    __tablename__ = 'blocks'
    
    id = Column(Integer, primary_key=True)
    blocks = Column(Integer)
    

class Column_(Base):
    __tablename__ = 'columns'
    
    id = Column(Integer, primary_key=True)
    columns = Column(Integer)
    
class Round(Base):
    __tablename__ = 'rounds'
    
    id = Column(Integer, primary_key=True)
    rounds = Column(Integer)

class Sponge(Base):
    __tablename__ = 'sponges'
    
    id = Column(Integer, primary_key=True)
    sponge = Column(String)

class TimeCost(Base):
    __tablename__ = 'tcosts'
    
    id = Column(Integer, primary_key=True)
    tcost = Column(Integer)
    
class MemoryCost(Base):
    __tablename__ = 'mcosts'
    
    id = Column(Integer, primary_key=True)
    mcost = Column(Integer)

class Result(Base):
    __tablename__ = 'results'
    
    id = Column(Integer, primary_key=True)
    
    elapsed = Column(Float)
    pss_mem = Column(BigInteger)
    uss_mem = Column(BigInteger)
    
    project_id = Column(Integer, ForeignKey('projects.id'))
    
    password_id = Column(Integer, ForeignKey('passwords.id'))
    salt_id = Column(Integer, ForeignKey('salts.id'))
    klen_id = Column(Integer, ForeignKey('klens.id'))
    
    round_id = Column(Integer, ForeignKey('rounds.id'))
    block_id = Column(Integer, ForeignKey('blocks.id'))
    
    column_id = Column(Integer, ForeignKey('columns.id'))
    sponge_id = Column(Integer, ForeignKey('sponges.id'))
    
    tcost_id = Column(Integer, ForeignKey('tcosts.id'))
    mcost_id = Column(Integer, ForeignKey('mcosts.id'))
    
    project = relationship('Project', back_populates='results')
    
    password = relationship('Password', back_populates='results')
    salt = relationship('Salt', back_populates='results')
    klen = relationship('Klen', back_populates='results')
    
    rounds = relationship('Round', back_populates='results')
    blocks = relationship('Block', back_populates='results')
    columns = relationship('Column_', back_populates='results')
    
    sponge = relationship('Sponge', back_populates='results')
    
    tcost = relationship('TimeCost', back_populates='results')
    mcost = relationship('MemoryCost', back_populates='results')

In [ ]:
Project.results = relationship('Result', back_populates='project')
Password.results = relationship('Result', back_populates='password')
Salt.results = relationship('Result', back_populates='salt')
Klen.results = relationship('Result', back_populates='klen')
Round.results = relationship('Result', back_populates='rounds')
Block.results = relationship('Result', back_populates='blocks')
Column_.results = relationship('Result', back_populates='columns')
Sponge.results = relationship('Result', back_populates='sponge')

TimeCost.results = relationship('Result', back_populates='tcost')
MemoryCost.results = relationship('Result', back_populates='mcost')

In [ ]:
engine = create_engine('sqlite:///measurements.db')
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)

In [ ]:
session = Session()

# Prepare the data

In [ ]:
projects = ['lyra2-c', 'lyra2-java']

In [ ]:
passwords, salts, klens = ['password'], ['salt'], [10]

In [ ]:
tcosts = [1, 20, 40, 60, 80, 100]

In [ ]:
mcosts = [3, 20, 40, 60, 80, 100]

In [ ]:
for project in projects:
    try:
        session.query(Project).filter_by(project=project).one()
    except NoResultFound:
        session.add(Project(project=project))

In [ ]:
for password in passwords:
    try:
        session.query(Password).filter_by(password=password).one()
    except NoResultFound:
        session.add(Password(password=password))

In [ ]:
for salt in salts:
    try:
        session.query(Salt).filter_by(salt=salt).one()
    except NoResultFound:
        session.add(Salt(salt=salt))

In [ ]:
for klen in klens:
    try:
        session.query(Klen).filter_by(klen=klen).one()
    except NoResultFound:
        session.add(Klen(klen=klen))

In [ ]:
for tcost in tcosts:
    try:
        session.query(TimeCost).filter_by(tcost=tcost).one()
    except NoResultFound:
        session.add(TimeCost(tcost=tcost))

In [ ]:
for mcost in mcosts:
    try:
        session.query(MemoryCost).filter_by(mcost=mcost).one()
    except NoResultFound:
        session.add(MemoryCost(mcost=mcost))

In [ ]:
for blocks in params['matrix']['blocks']:
    try:
        session.query(Block).filter_by(blocks=blocks).one()
    except NoResultFound:
        session.add(Block(blocks=blocks))

In [ ]:
for rounds in params['matrix']['rounds']:
    try:
        session.query(Round).filter_by(rounds=rounds).one()
    except NoResultFound:
        session.add(Round(rounds=rounds))

In [ ]:
for columns in params['matrix']['columns']:
    try:
        session.query(Column_).filter_by(columns=columns).one()
    except NoResultFound:
        session.add(Column_(columns=columns))

In [ ]:
for sponge in params['matrix']['sponge']:
    [sponge, _] = compose_sponge_name(sponge)
    try:
        session.query(Sponge).filter_by(sponge=sponge).one()
    except NoResultFound:
        session.add(Sponge(sponge=sponge))

In [ ]:
session.commit()
session.close()

# Perform the measurements

In [ ]:
observer = ProcessObserver(ntimes=1, mtimes=5)

In [ ]:
matrix = params['matrix']

In [ ]:
def run_one_configuration(project, password, salt, klen, tcost, mcost, blocks, rounds, columns, sponge):
    
    session = Session()

    option = matrix['option']
    threads = matrix['threads']
    bench = matrix['bench']

    [sponge, _] = compose_sponge_name(sponge)

    name = compose_lyra2_name(
        option, threads, columns, sponge, rounds, blocks
    )
    
    if project == 'lyra2-c':
        executable = build_path0.joinpath(name)
    else:
        # Java implementataion accepts sponge type, number of rounds etc as runtime parameters
        executable = build_path1.joinpath('lyra2-1.3-SNAPSHOT-jar-with-dependencies.jar')
    
    project_id = session.query(Project).filter_by(project=project).one().id
    password_id = session.query(Password).filter_by(password=password).one().id
    salt_id = session.query(Salt).filter_by(salt=salt).one().id
    klen_id = session.query(Klen).filter_by(klen=klen).one().id
    round_id = session.query(Round).filter_by(rounds=rounds).one().id
    block_id = session.query(Block).filter_by(blocks=blocks).one().id
    column_id = session.query(Column_).filter_by(columns=columns).one().id
    sponge_id = session.query(Sponge).filter_by(sponge=sponge).one().id
    tcost_id = session.query(TimeCost).filter_by(tcost=tcost).one().id
    mcost_id = session.query(MemoryCost).filter_by(mcost=mcost).one().id

    try:
        result = session.query(Result).filter_by(
            project_id=project_id
            , password_id=password_id
            , salt_id=salt_id
            , klen_id=klen_id
            , round_id=round_id
            , block_id=block_id
            , column_id=column_id
            , sponge_id=sponge_id
            , tcost_id=tcost_id
            , mcost_id=mcost_id
        ).one()

        print('(cached) ' + project + ': ' + name)
        print((result.elapsed, result.pss_mem, result.uss_mem))

    except NoResultFound:
        if project == 'lyra2-c':
            elapsed, pss_mem, uss_mem = observer.run([
                executable
                , password
                , salt
                , str(klen)
                , str(tcost)
                , str(mcost)
            ])
        else:
            elapsed, pss_mem, uss_mem = observer.run([
                'java'
                , '-jar'
                , str(executable)
                , '--blocks', str(blocks)
                , '--columns', str(columns)
                , '--full-rounds', str(rounds)
                , '--sponge', sponge
                , password
                , salt
                , str(klen)
                , str(tcost)
                , str(mcost)
            ])

        session.add(Result(
            elapsed=elapsed
            , pss_mem=pss_mem
            , uss_mem=uss_mem
            , project_id=project_id
            , password_id=password_id
            , salt_id=salt_id
            , klen_id=klen_id
            , round_id=round_id
            , block_id=block_id
            , column_id=column_id
            , sponge_id=sponge_id
            , tcost_id=tcost_id
            , mcost_id=mcost_id
        ))
                
        session.commit()
        session.close()

        print('(new) ' + project + ': ' + name)
        print((elapsed, pss_mem, uss_mem))

In [ ]:
with multiprocessing.Pool(4) as pool:
    pool.starmap(run_one_configuration, itertools.product(projects, passwords, salts, klens, tcosts, mcosts, matrix['blocks'], matrix['rounds'], matrix['columns'], matrix['sponge']), 10)